In [6]:
import pandas as pd

train_path = "file/HospitalDataSet1/data_cat10_annotated_train.txt"
eval_path = "file/HospitalDataSet1/data_cat10_annotated_eval.txt"
test_path = "file/HospitalDataSet1/data_cat10_annotated_test.txt"
train_output = "file/train.json"
eval_output = "file/eval.json"
test_output = "file/test.json"

In [7]:
import ml.config as config
import json

id2label = {}
with open('file/id2label.txt', 'r') as txt:
    for line in txt:
        ID, label = line.strip().split(' ')
        id2label[ID] = label

print(id2label)
with open('file/id2label.json', 'w') as f:
    json.dump(id2label, f, ensure_ascii=False, indent = 2)

label2id = {v:k for k, v in id2label.items()}
print(label2id)
with open('file/label2id.json', 'w') as f:
    json.dump(label2id, f, ensure_ascii=False, indent = 2)

{'1': '男科', '2': '皮肤科', '3': '其它', '4': '妇科', '5': '心血管内科', '6': '耳鼻喉科', '7': '消化内科', '8': '小儿内科', '9': '呼吸内科', '10': '内分泌科', '11': '神经内科'}
{'男科': '1', '皮肤科': '2', '其它': '3', '妇科': '4', '心血管内科': '5', '耳鼻喉科': '6', '消化内科': '7', '小儿内科': '8', '呼吸内科': '9', '内分泌科': '10', '神经内科': '11'}


In [8]:
def get_data(in_path, output_path):
    data = {}
    with open(in_path, "r") as f:
        for i, line in enumerate(f.readlines()):
            line = line.strip()
            line = line.split("\t")
            if len(line) != 2:
                continue
            label, text = line[0], line[1]
            if label not in data:
                data[label] = []
            data[label].append(text)
    for key in data.keys():
        print("label: %s, num: %d" % (id2label[key], len(data[key])))
    print("\n")
    with open(output_path, mode="w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [9]:
get_data(train_path, train_output)
get_data(eval_path, eval_output)
get_data(test_path, test_output)

label: 心血管内科, num: 1195
label: 小儿内科, num: 2050
label: 其它, num: 2916
label: 男科, num: 10613
label: 妇科, num: 1305
label: 耳鼻喉科, num: 912
label: 呼吸内科, num: 2443
label: 消化内科, num: 1501
label: 内分泌科, num: 1144
label: 神经内科, num: 1191
label: 皮肤科, num: 3103


label: 心血管内科, num: 67
label: 内分泌科, num: 63
label: 呼吸内科, num: 137
label: 男科, num: 591
label: 皮肤科, num: 173
label: 神经内科, num: 66
label: 消化内科, num: 84
label: 妇科, num: 72
label: 其它, num: 162
label: 小儿内科, num: 114
label: 耳鼻喉科, num: 51


label: 消化内科, num: 83
label: 神经内科, num: 67
label: 心血管内科, num: 66
label: 皮肤科, num: 172
label: 男科, num: 591
label: 内分泌科, num: 64
label: 小儿内科, num: 114
label: 其它, num: 163
label: 呼吸内科, num: 136
label: 耳鼻喉科, num: 51
label: 妇科, num: 73




In [26]:
def data_to_csv(filepath):
    samples = []
    nanke = 0
    with open(filepath, 'r') as txt:
        for line in txt:
            ID, text = line.strip().split('\t')
            text = text.replace("||", "")
            label = id2label[ID]
            # if label == "男科":
            #     if nanke >= 2000:
            #         continue
            #     else:
            #         nanke += 1
            sample = label+'\t'+text
            samples.append(sample)

    outfile = config.train_data_file
    if 'eval' in filepath:
        outfile = config.eval_data_file
    if 'test' in filepath:
        outfile = config.test_data_file

    with open(outfile, 'w') as csv:
        csv.write('label\ttext\n')
        for sample in samples:
            csv.write(sample)
            csv.write('\n')

    if 'train' in filepath:
        print("训练数据：%s" % str(len(samples)))
    elif 'test' in filepath:
        print("测试数据：%s" % str(len(samples)))
    else:
        print("验证数据：%s" % str(len(samples)))


In [27]:
data_to_csv(config.train_raw_file)
data_to_csv(config.eval_raw_file)
data_to_csv(config.test_raw_file)

训练数据：28373
验证数据：1580
测试数据：1580


In [12]:
data = pd.read_csv(config.train_data_file, sep="\t")

In [13]:
data = data.sample(frac=1).reset_index(drop=True)
data

,label,text
0,耳鼻喉科,流鼻涕 大概 有 NUM 个 月 了 不断 吃 了 中药 也 不管 事 昨天 检查 说...
1,男科,你好 我 现在 早泄 腰腿 酸软 无力
2,其它,医生 您好 我 NUM 年 吃 抗抑郁 药物 以前 都 是 去 医院 拿药 的 现在 看...
3,妇科,经期 不 稳定 延迟 内分泌 失调
4,内分泌科,您好 我 给 我 老公 开药 他 现在 空腹血糖 NUM 餐后 两 小时 NUM 没...
...,...,...
19755,小儿内科,出汗 后 着凉 夜间 咳嗽 三天 导致 扁桃体 肿大 发炎 伴 轻微 流涕
19756,呼吸内科,肺炎 住院 了 出院 恢复 期 大夫 让 买 这 药 吃吃
19757,皮肤科,湿疹 多年 ( 有的 医生 说 是 皮炎 ) 夏季 复发 冬季 就 好了 ！ 主要 在 ...
19758,小儿内科,吹 空调 后 咳嗽 流涕 暂时 没有 痰 音 精神 状况 良好


In [14]:
keep = {"神经科":0, "心血管科":0, "呼吸科":0, "内分泌科":0, "消化科":0}
with open('file/Chinese-medical-dialogue-data/Data_数据/IM_内科/内科5000-33000.csv', mode="r", encoding="gbk", errors='ignore') as f:
    labels = []
    asklist = []
    labelset = set()
    for i in range(500000):
        lin = f.readline()[0:-1].split(',')
        if i==0:
            continue        
        #print(lin)
        if len(lin) == 4:
            if len(lin[1]+','+lin[2])<=128:
                if lin[0] in keep:
                    if keep[lin[0]] > 12000:
                        continue 
                    keep[lin[0]] += 1
                    labels.append(lin[0])
                    asklist.append(lin[1]+','+lin[2])
labels, asklist

(['心血管科',
  '心血管科',
  '心血管科',
  '内分泌科',
  '内分泌科',
  '内分泌科',
  '内分泌科',
  '心血管科',
  '消化科',
  '心血管科',
  '心血管科',
  '心血管科',
  '心血管科',
  '神经科',
  '心血管科',
  '心血管科',
  '内分泌科',
  '心血管科',
  '心血管科',
  '心血管科',
  '心血管科',
  '心血管科',
  '心血管科',
  '内分泌科',
  '心血管科',
  '心血管科',
  '心血管科',
  '心血管科',
  '心血管科',
  '心血管科',
  '心血管科',
  '内分泌科',
  '心血管科',
  '心血管科',
  '心血管科',
  '内分泌科',
  '心血管科',
  '心血管科',
  '心血管科',
  '心血管科',
  '心血管科',
  '心血管科',
  '内分泌科',
  '心血管科',
  '心血管科',
  '心血管科',
  '内分泌科',
  '呼吸科',
  '内分泌科',
  '内分泌科',
  '内分泌科',
  '内分泌科',
  '内分泌科',
  '内分泌科',
  '心血管科',
  '心血管科',
  '心血管科',
  '呼吸科',
  '心血管科',
  '呼吸科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',
  '神经科',


In [15]:

# print(asklist)
import json
import jieba
import pandas as pd
data = pd.read_csv(config.train_data_file, sep="\t")
stopwords =[]
with open("ml/data/stopwords.txt", "r") as f:
    for line in f.readlines():
        line = line.strip()
        stopwords.append(line)
asklist = [" ".join([word for word in jieba.cut(sentence) if word not in stopwords]) for sentence in asklist]
buchong1 = pd.DataFrame(zip(labels, asklist), columns=['label', 'text'])
renamed = {"神经科":"神经内科", "心血管科":"心血管内科", "呼吸科":"呼吸内科", "内分泌科":"内分泌科", "消化科":"消化内科"}
buchong1['label'] = buchong1['label'].apply(lambda x: renamed[x])
# print(data.shape)
# data = pd.concat([buchong, data])
# data = data.sample(frac=1).reset_index(drop=True)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/zq/qvd6j_rd7mg0clbtsy_wzb2h0000gn/T/jieba.cache
Loading model cost 0.645 seconds.
Prefix dict has been built successfully.


In [16]:
buchong1

,label,text
0,心血管内科,高血压 患者 吃 党参 高血压 两天 女婿 党参 泡水 喝 您好 高血压 吃 党参
1,心血管内科,高血压 治疗 一位 中学教师 平时 身体健康 学校 组织 健康检查 发觉 高血压 去年 情况...
2,心血管内科,老年人 高血压 治疗 爷爷 68 年纪 高血压 领着 病 老人 痛苦 每次 喝药 控制 住 ...
3,内分泌科,糖尿病 还会 遗传 糖尿病 隔代遗传 我妈 糖尿病 很多年 没养 妹子 儿子 二十岁 没什么...
4,内分泌科,糖尿病 治疗 我妈 定期检查 仔细检查 仔细检查 出患 糖尿病 糖尿病 治疗 知晓 控制 糖...
...,...,...
60000,内分泌科,甲亢 吃 中药 治疗 甲亢 甲状腺 峡部 增厚 形态 饱满 实质 回声 粗糙 均匀 其内 未...
60001,内分泌科,母亲 甲亢 多年 未 治愈 反复 你好 请问 医院 治疗 甲亢 效果 母亲 因得 甲亢 多年...
60002,内分泌科,甲亢 药物 治疗 康复 周期 甲亢 吃药 吃 多长时间 治好
60003,内分泌科,甲亢 诱发 原因 朋友 关系 前不久 陪 医院 检查 医生 说 甲亢 想 问 治疗 效果 引...


In [17]:
import numpy as np
import pandas as pd
with open('file/Chinese-medical-dialogue-data/Data_数据/OAGD_妇产科/妇产科6-28000.csv', mode="r", encoding="gbk", errors='ignore') as f:
    labels = []
    asklist = []
    i = 0
    lines = f.readlines()[:50000]
    np.random.shuffle(lines)
    for i, lin in enumerate(lines):
        if i==0:
            continue
        if i > 15000:
            break
        lin = lin.split(',')
        if len(lin) == 4:
            if len(lin[1]+','+lin[2])<=128:
                labels.append(lin[0])
                asklist.append(lin[1]+','+lin[2])

asklist = [" ".join([word for word in jieba.cut(sentence) if word not in stopwords]) for sentence in asklist]
buchong2 = pd.DataFrame(zip(labels, asklist), columns=['label', 'text'])
buchong2['label'] = buchong2['label'].apply(lambda x: "妇科")
buchong2

,label,text
0,妇科,卵巢 早衰 稳妥 女 27 岁 医院 仔细检查 卵巢 早衰 备感 特别 担心 年轻 您好
1,妇科,输卵管 粘连 治疗 小腹 隐隐 疼 结婚 多年 怀孕 医院 检查 说 输卵管 堵 请问 输卵...
2,妇科,更年期 同房 避孕 女 48 岁 半个 月 胸闷 气短 一到 下午 烦躁 热 睡眠 不是太好...
3,妇科,康复 盆腔炎 身体素质 真的 差 生了病 盆腔炎 感觉 这辈子 妇科疾病 真的 有缘 复发 ...
4,妇科,白带 增多 病因 办法 解决 难受 困扰 久 舒服 求 解决办法 原因 引来 白带 增多
...,...,...
12514,妇科,孕期 宫颈炎 怎么弄 医生 您好 27 岁 怀孕 六个月 胎儿 情况 稳定 发现自己 宫颈炎...
12515,妇科,子宫 肌瘤 吃 紫菜 身体 好处 喜欢 吃 紫菜 医院 仔细检查 子宫 肌瘤 子宫 肌瘤 吃 紫菜
12516,妇科,想 问下 小儿 外阴炎 反复 发作 孩子 阴部 长 红 疹子 病发 禁不住 挠痒 地方 红肿...
12517,妇科,多发 宫颈 囊肿 B超 易发 宫颈 囊肿 吃 中药 西药 效果 治愈


In [18]:
import numpy as np
import pandas as pd
keep = {'耳鼻喉科':0, "内科":0}
renamed = {'耳鼻喉科':'耳鼻喉科', "内科":'小儿内科'}
with open('file/Chinese-medical-dialogue-data/Data_数据/Pediatric_儿科/儿科5-14000.csv', mode="r", encoding="gbk", errors='ignore') as f:
    labels = []
    asklist = []
    i = 0
    lines = f.readlines()
    np.random.shuffle(lines)
    for i, lin in enumerate(lines):
        if i==0:
            continue
        lin = lin.split(',')
        if len(lin) == 4:
            if lin[0] in keep:
                if keep[lin[0]] > 12000:
                    continue 
                if len(lin[1]+','+lin[2])<=128:
                    keep[lin[0]] += 1
                    labels.append(lin[0])
                    asklist.append(lin[1]+','+lin[2])

asklist = [" ".join([word for word in jieba.cut(sentence) if word not in stopwords]) for sentence in asklist]
buchong3 = pd.DataFrame(zip(labels, asklist), columns=['label', 'text'])
buchong3['label'] = buchong3['label'].apply(lambda x: renamed[x])
buchong3, buchong3['label'].value_counts()

(      label                                               text
 0      耳鼻喉科  小儿 扁桃体炎 症疼 诊治 男孩子 岁 说 咽 东西 疼 察觉到 吞咽 好像 困难 发烧 请...
 1      小儿内科                                            宝宝 拉白 屎
 2      耳鼻喉科  孩子 扁桃体 炎症 低烧 医治 我家 孩子 男 宝宝 岁 说话 时 嗓子疼 发现 吞咽 好像...
 3      耳鼻喉科  孩子 扁桃体 炎症 疼 医治 女 宝宝 月 嗓子眼 有点痛 察觉到 咳嗽 剧烈 好像 低烧 ...
 4      耳鼻喉科                                  八个 月 宝宝 鼻塞 流鼻涕 咳嗽
 ...     ...                                                ...
 23997  耳鼻喉科  幼儿 扁桃体炎 舒服 吃 食物 女 宝宝 岁 说 嗓子 喝水 疼 察觉到 咳嗽 好像 比较严...
 23998  耳鼻喉科  幼儿 扁桃体炎 症咽 痛 治疗 男宝 刚满 月 说 嗓子 喝水 疼 察觉到 吞咽 好像 困难...
 23999  耳鼻喉科  孩子 扁桃体 发炎 发生 常见 症状 女 宝宝 月 说 咽 东西 疼 发现 说话 声音 沙哑...
 24000  耳鼻喉科  儿童 中耳炎 听不清 说话 应 诊治 女 宝宝 岁 几天 孩子 说 耳朵 疼 察觉到 耳屎 ...
 24001  耳鼻喉科  小孩子 扁桃体 发炎 食欲 饮食 需注意 我家 女宝 岁 嗓子眼 有点痛 发现 咳嗽 剧烈 ...
 
 [24002 rows x 2 columns],
 耳鼻喉科    12001
 小儿内科    12001
 Name: label, dtype: int64)

In [19]:
import numpy as np
import pandas as pd
with open('file/Chinese-medical-dialogue-data/Data_数据/Andriatria_男科/男科5-13000.csv', mode="r", encoding="gbk", errors='ignore') as f:
    labels = []
    asklist = []
    i = 0
    lines = f.readlines()[:50000]
    np.random.shuffle(lines)
    for i, lin in enumerate(lines):
        if i==0:
            continue
        if i > 5000:
            break
        lin = lin.split(',')
        if len(lin) == 4:
            if len(lin[1]+','+lin[2])<=128:
                labels.append(lin[0])
                asklist.append(lin[1]+','+lin[2])

asklist = [" ".join([word for word in jieba.cut(sentence) if word not in stopwords]) for sentence in asklist]
buchong4 = pd.DataFrame(zip(labels, asklist), columns=['label', 'text'])
buchong4['label'] = buchong4['label'].apply(lambda x: "男科")
buchong4, buchong4['label'].value_counts()

(     label                                               text
 0       男科  前列腺 处 增生 治疗 男性 前列腺 增生 治疗 前列腺 增生 年 尿频 厉害 治疗 男性 ...
 1       男科  患 早泄 治疗 改善 手淫 挺 频繁 家里 介绍 对象 挺 发现 好像 早泄 手淫 导致 真...
 2       男科  缓解 性功能 障碍 喜欢 老婆 房事 老婆 很小 气 礼拜 三次 房事 勃起 推拿 没什么 ...
 3       男科                      治疗 前列腺 增生 治疗 前列腺 增生 治疗 前列腺 增生
 4       男科                                前列腺 肥大 检查 前要 前列腺 肥大
 ...    ...                                                ...
 4333    男科  患 精索 静脉曲张 感觉 阴囊 局部 持续 间歇 坠胀 疼痛感 隐痛 钝 痛 向下 腹部 检...
 4334    男科          男人 不育 病因 精液 异常 活跃度 差 说 不育症 心碎 请问 男性 不育 病因
 4335    男科      精索 静脉曲张 2.6 治疗 叔叔 年轻 干 体力 静脉 曲 精索 静脉曲张 2.6 治疗
 4336    男科    治疗 射精 费用 手淫 习惯 发现 手淫 特别 射 性生活 担心 早泄 请问 治疗 射精 费用
 4337    男科  性功能 障碍 关系 男女之间 那点 事 很感兴趣 妻子 离婚 长时间 那种 事情 却是 爱 ...
 
 [4338 rows x 2 columns],
 男科    4338
 Name: label, dtype: int64)

In [29]:
data = pd.read_csv(config.train_data_file, sep="\t")
data = pd.concat([data, buchong1, buchong2, buchong3, buchong4])
data = data.sample(frac=1).reset_index(drop=True)
data, data['label'].value_counts()

(        label                                               text
 0        呼吸内科  小孩 病毒性 肺炎 同学 家 孩子 住院 说 发烧 觉着 感冒 低 烧烧 抽 医院 仔细检查...
 1        神经内科  治疗 坐骨神经 痛 上班 比较忙 感觉 操劳 再次出现 臀部 疼痛 症状 大腿 痛 感觉 痛...
 2        消化内科  小肚子 胀气 吃 东西 嘴酸 肚子疼 分不清 胃 疼 肚子 发烧 咳嗽 疼痛 有时候 想 厕...
 3       心血管内科                   高血压 吃 板栗 血压 高 想 吃 板栗 您好 高血压 吃 板栗
 4        神经内科  面 神经炎 治疗 方法 睡午觉 嘴 眼下 垂 脸上 抽 抽 中锋 医院 说 面 神经炎 医生...
 ...       ...                                                ...
 129232   神经内科  羊羔 疯是 病 治疗 昨天 周末 正好 时间 姑姑家 姑姑 姑姑 亲戚 玩 昏倒 口吐白沫 ...
 129233     妇科       卵巢 早衰 稳妥 女 27 岁 医院 仔细检查 卵巢 早衰 备感 特别 担心 年轻 您好
 129234     妇科             乳头 不疼 怀孕 女 30 岁 怀孕 担心 怎么回事 请问 乳头 不疼 怀孕
 129235     妇科  患 宫颈 糜烂 后该 治 一段时间 小腹 疼 腰 酸 私处 舒服 瘙痒 烧灼 下午 真的 太...
 129236   神经内科           癫痫 发作 抑制 16 岁 孩子 次发 癫痫 检查 讲 脑 收缩 讲 一点 开裂
 
 [129237 rows x 2 columns],
 男科       14951
 呼吸内科     14444
 小儿内科     14051
 妇科       13824
 消化内科     13502
 心血管内科    13196
 神经内科     13192
 内分泌科     13145
 耳鼻喉科     12913
 皮肤科       3103
 其它        2916
 N

In [30]:
path = config.train_data_file.replace("1", "2")
path

'/Volumes/yyx/projects/Chinese-Text-Classification/ml/data/train2.csv'

In [31]:
data.to_csv(path, sep='\t', header=True)

In [10]:
import pandas as pd
data = pd.read_csv("ml/data/train2.csv", sep='\t', header=0, usecols=[1,2])
data1 = pd.read_csv("dermatology/dermatology.csv", sep=',', header=0)
data, data1

(        label                                               text
 0        呼吸内科  小孩 病毒性 肺炎 同学 家 孩子 住院 说 发烧 觉着 感冒 低 烧烧 抽 医院 仔细检查...
 1        神经内科  治疗 坐骨神经 痛 上班 比较忙 感觉 操劳 再次出现 臀部 疼痛 症状 大腿 痛 感觉 痛...
 2        消化内科  小肚子 胀气 吃 东西 嘴酸 肚子疼 分不清 胃 疼 肚子 发烧 咳嗽 疼痛 有时候 想 厕...
 3       心血管内科                   高血压 吃 板栗 血压 高 想 吃 板栗 您好 高血压 吃 板栗
 4        神经内科  面 神经炎 治疗 方法 睡午觉 嘴 眼下 垂 脸上 抽 抽 中锋 医院 说 面 神经炎 医生...
 ...       ...                                                ...
 129232   神经内科  羊羔 疯是 病 治疗 昨天 周末 正好 时间 姑姑家 姑姑 姑姑 亲戚 玩 昏倒 口吐白沫 ...
 129233     妇科       卵巢 早衰 稳妥 女 27 岁 医院 仔细检查 卵巢 早衰 备感 特别 担心 年轻 您好
 129234     妇科             乳头 不疼 怀孕 女 30 岁 怀孕 担心 怎么回事 请问 乳头 不疼 怀孕
 129235     妇科  患 宫颈 糜烂 后该 治 一段时间 小腹 疼 腰 酸 私处 舒服 瘙痒 烧灼 下午 真的 太...
 129236   神经内科           癫痫 发作 抑制 16 岁 孩子 次发 癫痫 检查 讲 脑 收缩 讲 一点 开裂
 
 [129237 rows x 2 columns],
      label                                               text
 0      皮肤科        皮肤过敏 全身痒有一年多，最近二个月特别 确诊什么病?需要用什么药来治疗?谢谢翁主任！
 1      皮肤科     爸今晚7点在公园做志愿者可能脚被虫咬，麻 晚7点，在公园值班，被啥虫咬，

In [11]:
import jieba

stopwords =[]
with open("ml/data/stopwords.txt", "r") as f:
    for line in f.readlines():
        line = line.strip()
        stopwords.append(line)

data1['text'] = data1['text'].apply(lambda sent: " ".join([word for word in jieba.cut(sent) if word not in stopwords]))
data1

,label,text
0,皮肤科,皮肤过敏 全身 痒 一年 二个月 特别 确诊 病 药来 治疗 谢谢 翁 主任
1,皮肤科,爸 今晚 点 公园 做 志愿者 脚 虫 咬 麻 晚 点 公园 值班 虫 咬 不疼 不痒 局部
2,皮肤科,湿疹 湿疹 明显好转 前 两年 膝盖 窝
3,皮肤科,面部 痤疮 面部 青春痘 已有 年 脸颊 下巴
4,皮肤科,右 手臂 半年 黑色 斑块 瘙痒 大腿 根部 内侧 长期存在 瘙痒 涂过 达克宁 无效 ...
...,...,...
9012,皮肤科,初起 时水 疱疹 痒 破 皮 湿疹 初起 时水 疱疹 痒 破 皮 医院 说 湿疹
9013,皮肤科,嘴周 湿疹 间断性 艾洛松 乳膏 痊愈 月 前 嘴 周红 起红点 医院 建议 用药 艾洛...
9014,皮肤科,白癜风 发现 眉毛 增了 一点 白斑 想 两天 陕西 榆林 疫情 有没有 影响 ...
9015,皮肤科,头上 穿掘性 毛囊炎 头上 囊肿 痒 痛 几天 感觉 头皮 疼痛 厉害


In [12]:
data = pd.concat([data, data1])
data = data.sample(frac=1).reset_index(drop=True)
data, data['label'].value_counts()

(       label                                               text
 0         其它  医生 您好  孩子 有 鼻炎  一直 用 糠酸莫米松 鼻 喷雾  无 不良 反  请 帮忙 ...
 1       呼吸内科      小儿 感冒 咳嗽 吃 孩子 一个劲 医院 详细 检查 感冒 喝药 治疗 想 小儿 感冒 吃
 2       神经内科         癫痫 患者 性功能 变差 用药 癫痫病 患者 吃药 十几年 性工能 越来越 不行 药
 3        皮肤科              脚上 长 跖 疣 三年   脚上 跖 疣 瘢痕 增生 体质 冷冻 激光治疗
 4       呼吸内科  肺 结节病 临床表现 治疗 方法 公司 联盟 员工 健康检查 查出来 肺 结节病 肺 结节病...
 ...      ...                                                ...
 138249    妇科  3pn 胚胎 怎么回事 前段时间 孕检 检查 受精卵 3pn 担心 宝宝 身体健康 威胁 害...
 138250  神经内科  请问 患上 神经衰弱 调节 医生 32 女 婚姻 中 致使 生气 脾气 不好 悲伤 自尊心 ...
 138251    男科  早泄  勃起 无力  NUM 年 有 过 前列腺炎  也 治好 了  用过 前列 舒乐  健...
 138252    妇科  做 宫颈 息肉 天上 厕所 有血 怎么回事 妈妈 56 做 宫颈 息肉 天 前 几天 说 厕...
 138253  内分泌科                                          请问 糖尿病 原因
 
 [138254 rows x 2 columns],
 男科       14951
 呼吸内科     14444
 小儿内科     14051
 妇科       13824
 消化内科     13502
 心血管内科    13196
 神经内科     13192
 内分泌科     13145
 耳鼻喉科     12913
 皮肤科      12120
 其它        2916
 Name: label, 

In [14]:
data.to_csv("DL/data/train_agg.csv", sep='\t', header=True, index=False)

In [1]:
import pandas as pd
train = pd.read_csv("DL/data/train_agg.csv", sep="\t", header=0)
eval = pd.read_csv("DL/data/eval.csv", sep="\t", header=0)
test = pd.read_csv("DL/data/test.csv", sep="\t", header=0)

data = pd.concat([train, eval, test])
data = data.sample(frac=1).reset_index(drop=True)
data, data['label'].value_counts()

(        label                                               text
 0          男科  练 太极拳 预防 前列腺 医生 你好 二十五岁 身边 朋友 患 前列腺炎 忧心 我会 患 病...
 1          男科                               万艾可 可以 和 立必劲 同时 服用 吗
 2         皮肤科  医院 诊断 过敏性 皮炎 家族史 病史 五月 患 湿疹 用药 好转 腿肚 感染 愈合 过敏性...
 3        内分泌科                    怀孕 期间 查出 甲亢 怀孕 中 查出 甲亢 治疗 胎儿 影响
 4        消化内科  慢性 胃炎 失眠 学生 本来 胃 不好 早上 吃饭 段时间 胃疼 学校 宠物医院 说 慢性 ...
 ...       ...                                                ...
 141409  心血管内科  原发性 血小板 增多 好多 年  一直 口服 羟基 脲片 控制  由于 购药 需 处方  烦...
 141410   耳鼻喉科  孩子 扁桃体 炎症 红肿 医治 女 宝宝 月 说 嗓子疼 察觉到 伴有 轻微 咳嗽 伴有 低...
 141411     男科  你好  李医生 ！ 我 手淫 落下 的 毛病  现在 睾丸 有时 有点 疼 ！ 射精无力  ...
 141412   神经内科  诊断 母猪 疯病 出门 治疗 段时间 出门 前段时间 诊断 出得 母猪 疯病 晕倒 浑身 抽...
 141413     妇科  患 外阴 白斑 治疗 私处 很痒 抓破 特别 疼 难受 医院 检查 说 外阴 白斑 真的 苦...
 
 [141414 rows x 2 columns],
 男科       16133
 呼吸内科     14717
 小儿内科     14279
 妇科       13969
 消化内科     13669
 心血管内科    13330
 神经内科     13325
 内分泌科     13272
 耳鼻喉科     13016
 皮肤科      12465
 其它        3239
 N

In [6]:
data = data[data.label != '其它']
n = data.shape[0]
eval_size = test_size = int(0.1 * n)
train_data = data.iloc[:(n-2*eval_size)]
eval_data = data.iloc[(n-2*eval_size):(n-eval_size)]
test_data = data.iloc[(n-eval_size):]
train_data, eval_data, test_data


(       label                                               text
 0         男科  练 太极拳 预防 前列腺 医生 你好 二十五岁 身边 朋友 患 前列腺炎 忧心 我会 患 病...
 1         男科                               万艾可 可以 和 立必劲 同时 服用 吗
 2        皮肤科  医院 诊断 过敏性 皮炎 家族史 病史 五月 患 湿疹 用药 好转 腿肚 感染 愈合 过敏性...
 3       内分泌科                    怀孕 期间 查出 甲亢 怀孕 中 查出 甲亢 治疗 胎儿 影响
 4       消化内科  慢性 胃炎 失眠 学生 本来 胃 不好 早上 吃饭 段时间 胃疼 学校 宠物医院 说 慢性 ...
 ...      ...                                                ...
 113160   皮肤科  局限 硬皮病   八年 前 局限性 硬皮病 治疗 三年   疫情 期间 隔离 帮 硬皮病 吃...
 113161  神经内科  脑 疝 昏迷 五天 瞳孔放大 康复 脑 疝 晕 迷 五天 瞳孔放大 插 呼吸机 颅内 出血 ...
 113162   皮肤科                      估计 股藓   时好时坏 抹 药能 缓解 反复 抹药 不见
 113163    妇科        输卵管 吻合 瘢痕 做 输卵管 吻合 手术 输卵管 瘢痕 瘢痕 会己 恢复 干扰 怀上
 113164  内分泌科  全身 发冷 治疗 懒得动 吃饭 一会 想 吃 东西 吃 不想 急死了 害怕 疾病 想 咨询 ...
 
 [110541 rows x 2 columns],
        label                                               text
 113165  呼吸内科                                  感冒 针灸 管用 感冒 针灸 管用
 113166   皮肤科  左 胳膊肘 处 皮肤 疙瘩 有时候 痒 有时候 不痒 脸部 眉毛 处 右腮部 皮肤过敏 色

In [7]:
print(train_data['label'].value_counts())
print(eval_data['label'].value_counts())
print(test_data['label'].value_counts())


男科       12966
呼吸内科     11776
小儿内科     11407
妇科       11086
消化内科     10954
心血管内科    10733
神经内科     10702
内分泌科     10569
耳鼻喉科     10430
皮肤科       9918
Name: label, dtype: int64
男科       1648
呼吸内科     1439
小儿内科     1435
妇科       1429
心血管内科    1325
消化内科     1319
内分泌科     1318
皮肤科      1312
耳鼻喉科     1308
神经内科     1284
Name: label, dtype: int64
男科       1519
呼吸内科     1502
妇科       1454
小儿内科     1437
消化内科     1396
内分泌科     1385
神经内科     1339
耳鼻喉科     1278
心血管内科    1272
皮肤科      1235
Name: label, dtype: int64


In [8]:
train_data.to_csv("DL/data/train_new.csv", sep='\t', header=True, index=False)
eval_data.to_csv("DL/data/eval_new.csv", sep='\t', header=True, index=False)
test_data.to_csv("DL/data/test_new.csv", sep='\t', header=True, index=False)